<a href="https://colab.research.google.com/github/alim98/MPI/blob/main/VGG_Final_Feature_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"

!unzip -q downloaded_file.zip

--2025-01-24 18:15:36--  https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.134.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264688649 (1.2G) [application/octet-stream]
Saving to: ‘downloaded_file.zip’

downloaded_file.zip 100%[===================>]   1.18G  69.0MB/s    in 18s     

2025-01-24 18:15:57 (65.7 MB/s) - ‘downloaded_file.zip’ saved [1264688649/1264688649]



In [ ]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00


# raw

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_dimensions(features_scaled):
    # Direct PCA
    pca_direct = PCA(n_components=2, random_state=42)
    pca_result = pca_direct.fit_transform(features_scaled)

    # PCA 50 + UMAP
    pca = PCA(n_components=50, random_state=42)
    features_pca = pca.fit_transform(features_scaled)
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    umap_result = umap_reducer.fit_transform(features_pca)

    # Direct UMAP
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return pca_result, umap_result, umap_direct_result

def create_plots():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    fig = make_subplots(
        rows=1, cols=3,  # Adjusted to one row with three columns
        subplot_titles=('VGG - Direct PCA', 'VGG - PCA50 + UMAP', 'VGG - Direct UMAP'),
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )

    model_name = 'VGG segmentation type_0 input_maskFalse'  # One model as per your request
    file_path = '/content/VGG_CSV_segmentation_typeavg_0_input_mask_False_features.csv'  # Update this path as needed

    # Read the model data
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    pca_result, umap_result, umap_direct_result = reduce_dimensions(features_scaled)

    reductions = [pca_result, umap_result, umap_direct_result]

    for col, reduction in enumerate(reductions, 1):
        for bbox in sorted(df['bbox_name'].unique()):  # Sort to ensure consistent order
            mask = df['bbox_name'] == bbox

            fig.add_trace(
                go.Scatter(
                    x=reduction[mask, 0],
                    y=reduction[mask, 1],
                    mode='markers',
                    name=bbox,
                    showlegend=True if col == 1 else False,
                    legendgroup=bbox,
                    marker=dict(
                        size=3.5,
                        color=color_mapping[bbox],
                        opacity=0.7
                    ),
                    text=df[mask]['Var1'],
                    hovertemplate=(
                        'Label: %{text}<br>'
                        'x: %{x:.2f}<br>'
                        'y: %{y:.2f}<br>'
                        'bbox: ' + bbox + '<br>'
                        '<extra></extra>'
                    )
                ),
                row=1, col=col  # Adjusted to fit into the new subplot structure
            )

    fig.update_layout(
        height=600,  # Adjusted height for a smaller layout
        width=1500,  # Adjusted width
        title_text="VGG segmentation type_0 input_maskFalse",
        showlegend=True,
    )

    methods = ['PCA', 'PCA50 + UMAP', 'Direct UMAP']
    for col in range(1, 4):
        fig.update_xaxes(title_text=f"{methods[col-1]} 1", row=1, col=col)
        fig.update_yaxes(title_text=f"{methods[col-1]} 2", row=1, col=col)

    return fig

def main():
    fig = create_plots()
    fig.show()

    fig.write_html("VGG segmentation type_0 input_maskFalse.html")
    print("Visualization has been saved as 'dimensionality_reduction_comparison.html'")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Visualization has been saved as 'dimensionality_reduction_comparison.html'


# Seg

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_dimensions(features_scaled):
    # Direct PCA
    pca_direct = PCA(n_components=2, random_state=42)
    pca_result = pca_direct.fit_transform(features_scaled)

    # PCA 50 + UMAP
    pca = PCA(n_components=50, random_state=42)
    features_pca = pca.fit_transform(features_scaled)
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    umap_result = umap_reducer.fit_transform(features_pca)

    # Direct UMAP
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return pca_result, umap_result, umap_direct_result

def create_plots():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    fig = make_subplots(
        rows=1, cols=3,  # Adjusted to one row with three columns
        subplot_titles=('VGG - Direct PCA', 'VGG - PCA50 + UMAP', 'VGG - Direct UMAP'),
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )

    model_name = 'VGG segmentation type_5 input_maskFalse'  # One model as per your request
    file_path = '/content/VGG_CSV_segmentation_type_5_input_mask_False_features.csv'  # Update this path as needed

    # Read the model data
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    pca_result, umap_result, umap_direct_result = reduce_dimensions(features_scaled)

    reductions = [pca_result, umap_result, umap_direct_result]

    for col, reduction in enumerate(reductions, 1):
        for bbox in sorted(df['bbox_name'].unique()):  # Sort to ensure consistent order
            mask = df['bbox_name'] == bbox

            fig.add_trace(
                go.Scatter(
                    x=reduction[mask, 0],
                    y=reduction[mask, 1],
                    mode='markers',
                    name=bbox,
                    showlegend=True if col == 1 else False,
                    legendgroup=bbox,
                    marker=dict(
                        size=3.5,
                        color=color_mapping[bbox],
                        opacity=0.7
                    ),
                    text=df[mask]['Var1'],
                    hovertemplate=(
                        'Label: %{text}<br>'
                        'x: %{x:.2f}<br>'
                        'y: %{y:.2f}<br>'
                        'bbox: ' + bbox + '<br>'
                        '<extra></extra>'
                    )
                ),
                row=1, col=col  # Adjusted to fit into the new subplot structure
            )

    fig.update_layout(
        height=600,  # Adjusted height for a smaller layout
        width=1500,  # Adjusted width
        title_text="VGG segmentation type_5 input_maskFalse",
        showlegend=True,
    )

    methods = ['PCA', 'PCA50 + UMAP', 'Direct UMAP']
    for col in range(1, 4):
        fig.update_xaxes(title_text=f"{methods[col-1]} 1", row=1, col=col)
        fig.update_yaxes(title_text=f"{methods[col-1]} 2", row=1, col=col)

    return fig

def main():
    fig = create_plots()
    fig.show()

    fig.write_html("VGG segmentation type_5 input_maskFalse.html")
    print("Visualization has been saved as 'dimensionality_reduction_comparison.html'")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Visualization has been saved as 'dimensionality_reduction_comparison.html'


# black

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_dimensions(features_scaled):
    # Direct PCA
    pca_direct = PCA(n_components=2, random_state=42)
    pca_result = pca_direct.fit_transform(features_scaled)

    # PCA 50 + UMAP
    pca = PCA(n_components=50, random_state=42)
    features_pca = pca.fit_transform(features_scaled)
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    umap_result = umap_reducer.fit_transform(features_pca)

    # Direct UMAP
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return pca_result, umap_result, umap_direct_result

def create_plots():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    fig = make_subplots(
        rows=1, cols=3,  # Adjusted to one row with three columns
        subplot_titles=('VGG - Direct PCA', 'VGG - PCA50 + UMAP', 'VGG - Direct UMAP'),
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )

    model_name = 'VGG segmentation type_5 input_mask True AVG'  # One model as per your request
    file_path = '/content/VGG_CSV_segmentation_type_5_input_mask_True_AVG_features.csv'  # Update this path as needed

    # Read the model data
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    pca_result, umap_result, umap_direct_result = reduce_dimensions(features_scaled)

    reductions = [pca_result, umap_result, umap_direct_result]

    for col, reduction in enumerate(reductions, 1):
        for bbox in sorted(df['bbox_name'].unique()):  # Sort to ensure consistent order
            mask = df['bbox_name'] == bbox

            fig.add_trace(
                go.Scatter(
                    x=reduction[mask, 0],
                    y=reduction[mask, 1],
                    mode='markers',
                    name=bbox,
                    showlegend=True if col == 1 else False,
                    legendgroup=bbox,
                    marker=dict(
                        size=3.5,
                        color=color_mapping[bbox],
                        opacity=0.7
                    ),
                    text=df[mask]['Var1'],
                    hovertemplate=(
                        'Label: %{text}<br>'
                        'x: %{x:.2f}<br>'
                        'y: %{y:.2f}<br>'
                        'bbox: ' + bbox + '<br>'
                        '<extra></extra>'
                    )
                ),
                row=1, col=col  # Adjusted to fit into the new subplot structure
            )

    fig.update_layout(
        height=600,  # Adjusted height for a smaller layout
        width=1500,  # Adjusted width
        title_text="VGG segmentation type_5 input_mask True AVG",
        showlegend=True,
    )

    methods = ['PCA', 'PCA50 + UMAP', 'Direct UMAP']
    for col in range(1, 4):
        fig.update_xaxes(title_text=f"{methods[col-1]} 1", row=1, col=col)
        fig.update_yaxes(title_text=f"{methods[col-1]} 2", row=1, col=col)

    return fig

def main():
    fig = create_plots()
    fig.show()

    fig.write_html("VGG segmentation type_5 input_mask True AVG.html")
    print("Visualization has been saved as 'dimensionality_reduction_comparison.html'")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Visualization has been saved as 'dimensionality_reduction_comparison.html'


#AVG

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_dimensions(features_scaled):
    # Direct PCA
    pca_direct = PCA(n_components=2, random_state=42)
    pca_result = pca_direct.fit_transform(features_scaled)

    # PCA 50 + UMAP
    pca = PCA(n_components=50, random_state=42)
    features_pca = pca.fit_transform(features_scaled)
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    umap_result = umap_reducer.fit_transform(features_pca)

    # Direct UMAP
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return pca_result, umap_result, umap_direct_result

def create_plots():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    fig = make_subplots(
        rows=1, cols=3,  # Adjusted to one row with three columns
        subplot_titles=('VGG - Direct PCA', 'VGG - PCA50 + UMAP', 'VGG - Direct UMAP'),
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )

    model_name = 'VGG segmentation type_5 input_mask True Black'  # One model as per your request
    file_path = '/content/VGG_CSV_segmentation_type_5_input_mask_True_Black_Black_features.csv'  # Update this path as needed

    # Read the model data
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    pca_result, umap_result, umap_direct_result = reduce_dimensions(features_scaled)

    reductions = [pca_result, umap_result, umap_direct_result]

    for col, reduction in enumerate(reductions, 1):
        for bbox in sorted(df['bbox_name'].unique()):  # Sort to ensure consistent order
            mask = df['bbox_name'] == bbox

            fig.add_trace(
                go.Scatter(
                    x=reduction[mask, 0],
                    y=reduction[mask, 1],
                    mode='markers',
                    name=bbox,
                    showlegend=True if col == 1 else False,
                    legendgroup=bbox,
                    marker=dict(
                        size=3.5,
                        color=color_mapping[bbox],
                        opacity=0.7
                    ),
                    text=df[mask]['Var1'],
                    hovertemplate=(
                        'Label: %{text}<br>'
                        'x: %{x:.2f}<br>'
                        'y: %{y:.2f}<br>'
                        'bbox: ' + bbox + '<br>'
                        '<extra></extra>'
                    )
                ),
                row=1, col=col  # Adjusted to fit into the new subplot structure
            )

    fig.update_layout(
        height=600,  # Adjusted height for a smaller layout
        width=1500,  # Adjusted width
        title_text="VGG segmentation type_5 input_mask True Black",
        showlegend=True,
    )

    methods = ['PCA', 'PCA50 + UMAP', 'Direct UMAP']
    for col in range(1, 4):
        fig.update_xaxes(title_text=f"{methods[col-1]} 1", row=1, col=col)
        fig.update_yaxes(title_text=f"{methods[col-1]} 2", row=1, col=col)

    return fig

def main():
    fig = create_plots()
    fig.show()

    fig.write_html("VGG segmentation type_5 input_mask True Black.html")
    print("Visualization has been saved as 'dimensionality_reduction_comparison.html'")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



Visualization has been saved as 'dimensionality_reduction_comparison.html'
